In [ ]:
import numpy as np
from ngsolve.webgui import Draw
from ngsolve import CF
from fiberamp.fiber.microstruct.bragg import Bragg, BraggExact, BraggScalar, plotlogf
from scipy.optimize import newton

# Bragg Package Tutorial

The Bragg Package contains python classes allowing the user to model Bragg type fibers.  These fibers consist of nested rings of homogeneous material.  The classes allow for an arbitrary number of these rings.  One can find the exact fields associated with leaky and guided modes of these fibers, approximate fields based on smooth interface conditions, or use numerical methods to try and match the exact and/or approximate field.  There are 3 associated classes:

- BraggExact: Find exact leaky or guided modes based on Maxwell's Interface conditions.
- BraggScalar: Find scalar modes based on smooth interface conditions.
- Bragg: Approximated the modes numerically using methods from parent class ModeSolver.

All three classes build a mesh and allow the user to visualize the obtained solutions.

In [ ]:
Draw(Bragg().mesh)

# Creating an instance of a Bragg fiber class

In this section we describe the necessary inputs to create instances of classes in the Bragg fiber package:

### Common Requirements

All the classes require 4 lists (or arrays) of the same length:

- ts: Gives the (dimensional) thickness of each layer.  The first entry is the radius of the core, the second the thickness of the first non-core material layer.  These values will be non-dimensionalized by a provided scaling factor prior to being used to construct the mesh.  Can be a float or integer.  

- ns: Gives the refractive index at each layer.  The entries can be floats or functions of wavelength (if including dispersion).

- mats: Gives names for the materials at each index.  Entries should be strings.  For the Bragg class the last entry must be 'Outer' to satisfy parent class.

- maxh: Gives maxh for mesh in each layer.  These values are scaled by the non-dimensionalize radii before being used, so a maxh of .1 means the true used maxh at that layer is .1 times the non-dimensional radius of the inner radius of that layer.

If the above lists aren't the same length an error is raised.  

In addition to this, all classes require the following:

- scale: float giving length to use for non-dimensionalization.  This is typically taken to be the same as the core radius, i.e. ts[0].

- wl: wavelength.

- bcs: either None or list/array of strings giving names for the material interfaces.  If not provided a default naming system is used.  If provided and the class is Bragg, the last entry must be 'OuterCircle' to satisfy parent class.

- ref: integer giving number of refinements for mesh.

- curve: factor for mesh curvature.  Needs to be higher than order for numerical convergence.

All the above are set to defaults giving a glass ring in air.  For the Bragg class this requires an extra layer of PML.

### Class specific inputs:

- For BraggScalar and BraggExact:

    - no_mesh=True/False: If set to True no mesh is built.  This allows for computation of propagation constants without the overhead of building a mesh.
    
- For Bragg:

    - fan=True/False: If set to True the mesh is created with Perfectly absorbing conditions on the lower half and opens to air/PML on the top.  Used to see if the effects observed in ARF fibers are due to the air cladding interface.
    - beta_sq_plane=True/False:  If set to True looks for propagation constants using beta squared instead of Z squared.  Useful when truncating cladding for leaky modes that travel in non ambient material (like for a PBG fiber).


# BraggScalar: Finding approximate Modes

We begin by showing how to use the BraggScalar class to find approximate modes.  The inputs shown are the defaults


In [ ]:
A = BraggScalar(scale=5e-5,
                ts=[5e-5, 1e-5, 2e-5],
                ns=[1, 1.44, 1],
                mats=['air', 'glass', 'air'], 
                maxhs=[.2, .02, .08], 
                bcs=None, no_mesh=False,
                wl=1.2e-6, ref=0, curve=8)

### Attributes:

In [ ]:
A.__dict__

### Mesh and index function

In [ ]:
Draw(CF(list(A.ns)), A.mesh)

# Modefinding

BraggScalar finds modes semi-analytically.  It calculates the determinant of a matrix and the zeros of this determinant give the propagation constants.  From this the modes can be found.

We luckily know where modes are likely to be found.  If we are looking for leaky modes, their real parts tend to appear just below the the lowest value of the wavenumber.  The wavenumber is different for each material and can be calculated as

$k = k_0 n =(2\pi/\lambda)\, n$

where $\lambda$ is the input wavelength.  If we are looking for guided modes their real parts must appear between the lowest value of $k$ and the highest.  We calculate these here:

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale
k_high = A.k0 * A.ns[1] * A.scale
k_low, k_high

Note that e use non-dimensionalized values when modefinding. That's why we scaled by A.scale above.  

### Other important quantities for modefinding:

- nu:  Each mode varies sinusoidally in the azimuthal direction.  The number of waves in this behavior is given by $\nu$, here given as the variable nu.  

- outer: The outer behavior of a mode can be decaying (guided) or blowing up (leaky).  These behaviors are determined by which of two Hankel functions are used in the outer region.  We need to pick Hankel1 or Hankel2 ('h1' or 'h2')

We're looking for leaky modes, so we pick 'h2', and we want the fundamental, which for scalar modes means we want nu=0.

In [ ]:
outer = 'h2'
nu = 0

# Visualizing the complex eigenfunction

To find the propagation constants, we need to see where the zeros of the determinant function are.  We do this by pluggin in the function A.determinant into the utility function plotlogf.

In [ ]:
A.determinant?

In [ ]:
plotlogf?

Large scale plot:

In [ ]:
plotlogf(A.determinant,.995*k_low,1.0001*k_low, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

Fundamental mode:

In [ ]:
plotlogf(A.determinant,.9999*k_low,1.00001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

# Getting the constant

For this we use the newton solver from scipy:

In [ ]:
guess = np.array(.99995*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


# Visualizing Fields

Now that we have beta we can form the fields:

In [ ]:
U = A.all_fields(beta1, nu, outer)

The function U defined above is an ngsolve coefficient function:

In [ ]:
Draw(100*U, A.mesh)

## Matplot plotting utilities

We can also visualize the field using matplotlib utilities.

In [ ]:
FsA = A.fields_matplot(beta1, nu, outer)

Fs is now a dictionary with two functions, one for 2D plots:

In [ ]:
A.plot2D_contour(FsA['Ez'], figsize=(10,10))

And one for 1D plots

In [ ]:
%matplotlib notebook
fig, ax = A.plot1D(FsA['Ez_rad'], double_r=True, rlist=[400,10000,400], nu=nu, maxscale=True,
                  linewidth=1.5, color='k', figsize=(10,7))


# BraggExact

The process of finding modes and visualizing them in BraggExact is almost the same as in BraggScalar.  The primary difference is that now we get all components of the fields, so there is more to visualize

In [ ]:
B = BraggExact(scale=5e-5,
                ts=[5e-5, 1e-5, 2e-5],
                ns=[1, 1.44, 1],
                mats=['air', 'glass', 'air'], 
                maxhs=[.2, .015, .04], 
                bcs=None, no_mesh=False,
                wl=1.2e-6, ref=0, curve=8)

In [ ]:
k_low = B.k0 * B.ns[0] * B.scale
k_high = B.k0 * B.ns[1] * B.scale
k_low, k_high

Note that for the exact fundamental mode, the z component is what we find first.  It actually has variation 
in the azimuthal direction, and its nu value is 1

In [ ]:
outer = 'h2'
nu = 1 

In [ ]:
plotlogf(B.determinant,.9999*k_low,1.00001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

# Getting the constant

For this we use the newton solver from scipy:

In [ ]:
guess = np.array(.99995*k_low)

beta2 = newton(B.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta2, ". Residual of determinant: ", abs(B.determinant(beta2, nu, outer)))


# Visualizing Fields

Now that we have beta we can form the fields:

In [ ]:
FsB = B.all_fields(beta2, nu, outer)

The dictionary FsB has many functions we can visualize:

In [ ]:
FsB

We just show Ez and Etv here.

In [ ]:
Draw(FsB['Ez'], B.mesh)

The fine ripple in the glass takes a lot of mesh elements to visualize properly.  To get a clearer view, decrease the maxh in that region.  Careful though as if you make it too small it will take a long time to build the mesh and draw the pictures.


## Etv: vector visualization

The field Etv is complex.  If we pick the real or imaginary part we get a vector field we can view.  The input vectors in Draw gives the ability to put more and more vectors to see fine behavior.

In [ ]:
Draw(FsB['Etv'].real, B.mesh, vectors={'grid_size':150})

## Matplot plotting utilities


In [ ]:
fsB = B.fields_matplot(beta2, nu, outer)

fsB now has more in it too:

In [ ]:
fsB.keys()

In [ ]:
B.plot2D_contour(fsB['Ez'], figsize=(10,10))

And one for 1D plots

In [ ]:
fig, ax = A.plot1D(fsB['Ez_rad'], double_r=True, rlist=[400,10000,400], nu=nu, maxscale=True,
                  linewidth=1.5, color='k', figsize=(10,7))


### Streamline plot

For visualizing vector functions using Matplot we currently have the streamplot.  Adding a quiver plot would be nice

In [ ]:
mag = lambda x,y: np.sqrt(np.abs(fsB['Ex'](x,y))**2 + np.abs(fsB['Ey'](x,y))**2)

In [ ]:
fig, ax = B.plot2D_streamlines(fsB['Ex'], fsB['Ey'], contourfunc=mag, seed_nr=[2,2, 2], seed_ntheta=16, 
                               rho_linewidth=2, broken_streamlines=True,
                               maxlength=.3, plot_seed=False);

# Bragg Numerical

The class for finding modes numerically is called Bragg:

In [ ]:
C = Bragg(scale=5e-5, ts=[5e-5, 1e-5, 2e-5, 2e-5],
          mats=['air', 'glass', 'air', 'Outer'], ns=[1, 1.44, 1, 1],
          maxhs=[.2, .025, .08, .1], bcs=['r1', 'r2', 'R', 'OuterCircle'],
          wl=1.2e-6, ref=0,
          curve=8)

## Numerical Modefinding

This class no longer has a determinant function.  We need to find the propagation constants using contours around the eigenvalues we want in the complex plane.  Since we know the exact propagation constants from the previous classes, we can use these, but first we have to translate them into Z values, since the methods in modesolver work in the Z and Z^2 plane.

### Scalar Modes:

The scalar modefinder is called leakymode.  It uses Z values.  There is a method we can use to find these from the exact propagation constants:

In [ ]:
Z1_true = C.sqrZfrom(beta1/A.scale)**.5  # method sqrZfrom gives Z^2 from beta (need to descale beta first)
Z1_true

Now we can look near this for the numerical propagation constants.  Note that we might still need to play with the radius, order, number of vectors sought, number of quadrature points etc to find the mode.

In [ ]:
center = Z1_true
radius = .1

p = 3
_, y, _, _, _, _ = C.leakymode(p, nspan=4, npts=4,
                                    rad=radius,
                                    ctr=center,
                                    alpha=5,
                                    niterations=5,
                                    nrestarts=0)

In [ ]:
for f in y:
    Draw(f, C.mesh)

## Vectorial Modefinding

The method for vector modes is called leakyvecmode.  It uses Z^2 values:

In [ ]:
Z2_true = C.sqrZfrom(beta2/A.scale)
Z2_true

In [ ]:
center = Z2_true
radius = .1
nspan = 4
npts = 4
p = 0

_, _, Es, phis, _ = C.leakyvecmodes(p=p, ctr=center, rad=radius,
                                       alpha=5,
                                       rhoinv=.9,
                                       quadrule='ellipse_trapez_shift',
                                       nspan=nspan, npts=npts,
                                       niterations=5, nrestarts=0,
                                       stop_tol=1e-9)


In [ ]:
for e in Es:
    Draw(e.real, C.mesh, vectors={'grid_size' : 100})

In [ ]:
for phi in phis:
    Draw(phi, C.mesh)